### DEinstagram crawling to csv
   
- daily_gamsung.csv 
- gamsung.bnb.csv 
- gamsung_curation.csv 
- hi.stay.tour.csv 
- rest_behappyhere.csv 
- sookso.diary.csv 
- sookso.hada.csv

1. 숙소이름 추출  
2. 계정별 숙소 리스트 취합  
3. 통합 숙소리스트를 기준으로 content 병합  
4. like 수 합치기  
5. content 일괄 전처리(토큰화, 불용어제거)  

'''
1. 중복된 이름 검색
2. 개수가 1인거 이상 뽑아내기
3. 리스트만들기
4. 행가져와서 like집계, content, tag 리스트로 append
5. 차례대로 기존 행 지워내기
6. place 네이버 API 사용해서 대체
'''

### 최종 데이터 셋  
columns = 숙소명, content(통합), like(통합), place(api), tags(통합), 게시수(중복값)


In [3]:


def daily_gamsung_extract(x):
    p = re.compile('\"[가-힣\t\n\r\f\v\s\_가-힣]+\"')
    m = p.findall(x)
    try:
        return m[0][1:-1]
    except:
        return np.nan

def gamsung_bnb_extract(x):
    p = re.compile("[\'|\"][가-힣]+[\'|\"]") 
    m = p.findall(x)
    try:
        return m[0][1:-1]
    except :
        return np.nan

def gamsung_curation_extract(x):
    p = re.compile('\📍[가-힣]+') # 
    m = p.findall(x)
    try:
        return m[0][1:] 
    except :
        return ''

def hi_stay_tour_extract(x):
    p = re.compile("\#[가-힣]+") # 
    m = p.findall(x)
    try:
        return m[0][1:] 
    except :
        return ''

def rest_behappyhere_extract(x):
    p = re.compile('\#[가-힣]+')
    m = p.findall(x)
    try:
        return m[1][1:]
    except:
        return np.nan

def sookso_diary_extract(x):
    p = re.compile('\📍 [가-힣]+\s[가-힇]+|\📍[가-힣]+\s[가-힇]+')
    m = p.findall(x)
    try:
        return m[0].split(' ')[-1]
    except:
        return np.nan

def sookso_hada_extract(x):
    p = re.compile('\#[가-힣]+')
    m = p.findall(x)
    if  m[1] == '#숙소하다': m[1], m[0] = m[0], m[1]
    return m[1][1:]


def daily_gamsung_extract(x):
    p = re.compile('\"[가-힣\t\n\r\f\v\s\_가-힣]+\"')
    m = p.findall(x)
    try:
        return m[0][1:-1]
    except:
        return np.nan

# def convert(x):
#     return int(x.replace(',',''))

def convert(x):
    if x:
        return int(x.replace(',',''))
    else:
        return 0

#####################################################
### daily_gamsung(완료)    
def insta_spark_processing():
    from pyspark.sql.types import StructType,StructField, StringType, IntegerType
    from pyspark.sql.functions import array_contains, udf
    from pyspark.conf import SparkConf
    from pyspark.sql import SparkSession


    from datetime import datetime, timedelta
    from pymongo import MongoClient
    import pandas as pd
    import numpy as np
    from difflib import SequenceMatcher
    from collections import Counter
    from datetime import datetime
    import re

    import time
    import os
    import sys
    import urllib.request
    import json

    nowtime = datetime.today().strftime("%Y-%m-%d")#  - pd.DateOffset(days=1)
    spark = SparkSession\
            .builder\
            .appName('Python Spark SQL basic example')\
            .getOrCreate()

    sc = spark.sparkContext
    
    # pymongo connect
    client = MongoClient('localhost',27017) # mongodb 27017 port
    db = client.ojo_db
    cur = db['cluster']

    coronaStage = spark.read.parquet(f"hdfs://localhost:9000/data/corona/coronaStage_2021-10-04")

    daily_gamsung_DFP = spark.read.parquet(f"hdfs://localhost:9000/data/insta/daily_gamsung_{nowtime}.parquet")
    daily_gamsung_DFP = daily_gamsung_DFP.drop('Unnamed: 0','unix')
    daily_gamsung = daily_gamsung_DFP.toPandas()

    gamsung_bnb_DFP = spark.read.parquet(f"hdfs://localhost:9000/data/insta/gamsung_bnb_{nowtime}.parquet")
    gamsung_bnb_DFP = gamsung_bnb_DFP.drop('Unnamed: 0','unix')
    gamsung_bnb = gamsung_bnb_DFP.toPandas()

    gamsung_curation_DFP = spark.read.parquet(f"hdfs://localhost:9000/data/insta/gamsung_curation_{nowtime}.parquet")
    gamsung_curation_DFP = gamsung_curation_DFP.drop('Unnamed: 0','unix')
    gamsung_curation = gamsung_curation_DFP.toPandas()

    hi_stay_tour_DFP = spark.read.parquet(f"hdfs://localhost:9000/data/insta/hi_stay_tour_{nowtime}.parquet")
    hi_stay_tour_DFP = hi_stay_tour_DFP.drop('Unnamed: 0','unix')
    hi_stay_tour = hi_stay_tour_DFP.toPandas()

    rest_behappyhere_DFP = spark.read.parquet(f"hdfs://localhost:9000/data/insta/rest_behappyhere_{nowtime}.parquet")
    rest_behappyhere_DFP = rest_behappyhere_DFP.drop('Unnamed: 0','unix')
    rest_behappyhere = rest_behappyhere_DFP.toPandas()

    sookso_diary_DFP = spark.read.parquet(f"hdfs://localhost:9000/data/insta/sookso_diary_{nowtime}.parquet")
    sookso_diary_DFP = sookso_diary_DFP.drop('Unnamed: 0','unix')
    sookso_diary = sookso_diary_DFP.toPandas()

    sookso_hada_DFP = spark.read.parquet(f"hdfs://localhost:9000/data/insta/sookso_hada_{nowtime}.parquet")
    sookso_hada_DFP = sookso_hada_DFP.drop('Unnamed: 0','unix')
    sookso_hada = sookso_hada_DFP.toPandas()
    
    #######################################
    daily_gamsung_name = daily_gamsung.content.apply(lambda x : daily_gamsung_extract(x))
    daily_gamsung['name'] = daily_gamsung_name
    daily_gamsung = daily_gamsung[~(daily_gamsung['name'].isna())]

    # name data가 없는 index 지우기
    indexNames = daily_gamsung[daily_gamsung['name'] == ''].index
    daily_gamsung.drop(indexNames, inplace=True) 

    if daily_gamsung.like.dtype == 'O':
        daily_gamsung.like = daily_gamsung.like.apply(lambda x : convert(x))


    # 1,2,3
    dg_cnt = Counter(daily_gamsung.name)
    li =[]
    for name, cnt  in dg_cnt.items():
        if cnt > 1:
            li.append(name)   

    # 4. 행가져와서 like집계, content, tag list append
    for i in li:           # 중복된 name 반복
        ex = daily_gamsung.loc[daily_gamsung.name == i]  # 반복된 name의 행을 가져온다
        ex.reset_index(drop=True,inplace=True)     # index reset

        content = ex.iloc[0,:].content
        date = ex.iloc[0,:].date
        like = ex.iloc[0,:].like
        place = ex.iloc[0,:].place
        tags = ex.iloc[0,:].tags
        imgUrl = ex.iloc[0,:].imgUrl
        name = ex.name[0]

        for i in range(len(ex)-1):
            like += ex.iloc[i+1,:].like
            tags += ex.iloc[i+1,:].tags

        # 5.차례대로 기존 행 지워내기
        indexNames = daily_gamsung[daily_gamsung['name'] == ex.name[0]].index
        daily_gamsung.drop(indexNames, inplace=True) 
        daily_gamsung = daily_gamsung.append(pd.Series(data=[content,date,like,place,tags,imgUrl,name],index=daily_gamsung.columns),ignore_index=True)

    dg_tmp = []
    for i in daily_gamsung.name:
        dg_tmp.append(dg_cnt[i]-1)

    daily_gamsung['overlap'] = dg_tmp


    #####################################################
    ### gamsun.bnb
    gamsung_bnb_name = gamsung_bnb.content.apply(lambda x : gamsung_bnb_extract(x)) # gamsung_bnb_name = 
    gamsung_bnb['name'] = gamsung_bnb_name
    gamsung_bnb = gamsung_bnb[~(gamsung_bnb['name'].isna())]

    # name data가 없는 index 지우기
    indexNames = gamsung_bnb[gamsung_bnb['name'] == ''].index
    gamsung_bnb.drop(indexNames, inplace=True) 

    # like integet 변환 후 합치기

    if gamsung_bnb.like.dtype == 'O':
        gamsung_bnb.like = gamsung_bnb.like.apply(lambda x : convert(x))



    # 1, 중복된 이름 검색 
    # 2.개수가 1인거 이상 뽑아내기 
    # 3.리스트만들기
    gb_cnt = Counter(gamsung_bnb.name)      # Counter 모듈로 name data count
    li =[]
    for name, cnt  in gb_cnt.items():            # 중복된 것만 뽑기 (cnt가 1이상이면 중복)
        if cnt > 1:
            li.append(name)

    # 4. 행가져와서 like집계, content, tag list append
    for i in li:           # 중복된 name 반복
        ex = gamsung_bnb.loc[gamsung_bnb.name == i]  # 반복된 name의 행을 가져온다
        ex.reset_index(drop=True,inplace=True)     # index reset

        content = ex.iloc[0,:].content
        date = ex.iloc[0,:].date
        like = ex.iloc[0,:].like
        place = ex.iloc[0,:].place
        tags = ex.iloc[0,:].tags
        imgUrl = ex.iloc[0,:].imgUrl
        name = ex.name[0]

        for i in range(len(ex)-1):
            like += ex.iloc[i+1,:].like
            tags += ex.iloc[i+1,:].tags


        # 5.차례대로 기존 행 지워내기
        indexNames = gamsung_bnb[gamsung_bnb['name'] == ex.name[0]].index
        gamsung_bnb.drop(indexNames, inplace=True) 
        gamsung_bnb = gamsung_bnb.append(pd.Series(data=[content,date,like,place,tags,imgUrl,name],index=gamsung_bnb.columns),ignore_index=True)

    gb_tmp = []
    for i in gamsung_bnb.name:
        gb_tmp.append(gb_cnt[i]-1)


    gamsung_bnb['overlap'] = gb_tmp


    #####################################################
    ### gamsung_curation(완료)
    gamsung_curation_name = gamsung_curation.content.apply(lambda x : gamsung_curation_extract(x))
    gamsung_curation['name'] = gamsung_curation_name
    gamsung_curation = gamsung_curation[~(gamsung_curation['name'].isna())]


    # name data가 없는 index 지우기
    indexNames = gamsung_curation[gamsung_curation['name'] == ''].index
    gamsung_curation.drop(indexNames, inplace=True) 




    if gamsung_curation.like.dtype == 'O':
        gamsung_curation.like = gamsung_curation.like.apply(lambda x : convert(x))


    gc_cnt = Counter(gamsung_curation.name)      # Counter 모듈로 name data count
    li =[]
    for name, cnt  in gc_cnt.items():            # 중복된 것만 뽑기 (cnt가 1이상이면 중복)
        if cnt > 1:
            li.append(name)        


    # 4. 행가져와서 like집계, content, tag list append
    for i in li:           # 중복된 name 반복
        ex = gamsung_curation.loc[gamsung_curation.name == i]  # 반복된 name의 행을 가져온다
        ex.reset_index(drop=True,inplace=True)     # index reset

        content = ex.iloc[0,:].content
        date = ex.iloc[0,:].date
        like = ex.iloc[0,:].like
        place = ex.iloc[0,:].place
        tags = ex.iloc[0,:].tags
        imgUrl = ex.iloc[0,:].imgUrl
        name = ex.name[0]

        for i in range(len(ex)-1):
            like += ex.iloc[i+1,:].like
            tags += ex.iloc[i+1,:].tags

        # 5.차례대로 기존 행 지워내기
        indexNames = gamsung_curation[gamsung_curation['name'] == ex.name[0]].index
        gamsung_curation.drop(indexNames, inplace=True) 
        gamsung_curation = gamsung_curation.append(pd.Series(data=[content,date,like,place,tags,imgUrl,name],index=gamsung_curation.columns),ignore_index=True)

    gc_tmp = []
    for i in gamsung_curation.name:
        gc_tmp.append(gc_cnt[i]-1)

    gamsung_curation['overlap'] = gc_tmp

    #####################################################
    ### hi.stay.tour.csv (완료)
    hi_stay_tour_name = hi_stay_tour.content.apply(lambda x : hi_stay_tour_extract(x))
    hi_stay_tour['name'] = hi_stay_tour_name
    hi_stay_tour = hi_stay_tour[~(hi_stay_tour['name'].isna())]


    # name data가 없는 index 지우기
    indexNames = hi_stay_tour[hi_stay_tour['name'] == ''].index
    hi_stay_tour.drop(indexNames, inplace=True) 


    if hi_stay_tour.like.dtype == 'O':
        hi_stay_tour.like = hi_stay_tour.like.apply(lambda x : convert(x))


    # 1, 중복된 이름 검색 
    # 2.개수가 1인거 이상 뽑아내기 
    # 3.리스트만들기
    hst_cnt = Counter(hi_stay_tour.name)      # Counter 모듈로 name data count
    li =[]
    for name, cnt  in hst_cnt.items():            # 중복된 것만 뽑기 (cnt가 1이상이면 중복)
        if cnt > 1:
            li.append(name)  

    # 4. 행가져와서 like집계, content, tag list append
    for i in li:           # 중복된 name 반복
        ex = hi_stay_tour.loc[hi_stay_tour.name == i]  # 반복된 name의 행을 가져온다
        ex.reset_index(drop=True,inplace=True)     # index reset

        content = ex.iloc[0,:].content
        date = ex.iloc[0,:].date
        like = ex.iloc[0,:].like
        place = ex.iloc[0,:].place
        tags = ex.iloc[0,:].tags
        imgUrl = ex.iloc[0,:].imgUrl
        name = ex.name[0]

        for i in range(len(ex)-1):
            like += ex.iloc[i+1,:].like
            tags += ex.iloc[i+1,:].tags



        # 5.차례대로 기존 행 지워내기
        indexNames = hi_stay_tour[hi_stay_tour['name'] == ex.name[0]].index
        hi_stay_tour.drop(indexNames, inplace=True) 
        hi_stay_tour = hi_stay_tour.append(pd.Series(data=[content,date,like,place,tags,imgUrl,name],index=hi_stay_tour.columns),ignore_index=True)

    hst_cnt_tmp = []
    for i in hi_stay_tour.name:
        hst_cnt_tmp.append(hst_cnt[i]-1)

    hi_stay_tour['overlap'] = hst_cnt_tmp


    #####################################################
    ### rest_behappyhere.csv
    rest_behappyhere_name = rest_behappyhere.content.apply(lambda x : rest_behappyhere_extract(x))
    rest_behappyhere['name'] = rest_behappyhere_name
    rest_behappyhere = rest_behappyhere[~(rest_behappyhere['name'].isna())]

    # name data가 없는 index 지우기
    indexNames = rest_behappyhere[rest_behappyhere['name'] == ''].index
    rest_behappyhere.drop(indexNames, inplace=True) 



    if rest_behappyhere.like.dtype == 'O':
        rest_behappyhere.like = rest_behappyhere.like.apply(lambda x : convert(x))


    # 1, 중복된 이름 검색 
    # 2.개수가 1인거 이상 뽑아내기 
    # 3.리스트만들기
    rb_cnt = Counter(rest_behappyhere.name)      # Counter 모듈로 name data count
    li =[]
    for name, cnt  in rb_cnt.items():            # 중복된 것만 뽑기 (cnt가 1이상이면 중복)
        if cnt > 1:
            li.append(name)


    #####################################################
    # 4. 행가져와서 like집계, content, tag list append
    for i in li:           # 중복된 name 반복
        ex = rest_behappyhere.loc[rest_behappyhere.name == i]  # 반복된 name의 행을 가져온다
        ex.reset_index(drop=True,inplace=True)     # index reset

        content = ex.iloc[0,:].content
        date = ex.iloc[0,:].date
        like = ex.iloc[0,:].like
        place = ex.iloc[0,:].place
        tags = ex.iloc[0,:].tags
        imgUrl = ex.iloc[0,:].imgUrl
        name = ex.name[0]

        for i in range(len(ex)-1):
            like += ex.iloc[i+1,:].like
            tags += ex.iloc[i+1,:].tags



        # 5.차례대로 기존 행 지워내기
        indexNames = rest_behappyhere[rest_behappyhere['name'] == ex.name[0]].index
        rest_behappyhere.drop(indexNames, inplace=True) 
        rest_behappyhere = rest_behappyhere.append(pd.Series(data=[content,date,like,place,tags,imgUrl,name],index=rest_behappyhere.columns),ignore_index=True)

    rb_cnt_tmp = []
    for i in rest_behappyhere.name:
        rb_cnt_tmp.append(rb_cnt[i]-1)

    rest_behappyhere['overlap'] = rb_cnt_tmp

    #####################################################
    ### sookso.diary(완료)
    sookso_diary_name = sookso_diary.content.apply(lambda x : sookso_diary_extract(x))
    sookso_diary['name'] = sookso_diary_name
    sookso_diary = sookso_diary[~(sookso_diary['name'].isna())]


    # name data가 없는 index 지우기
    indexNames = sookso_diary[sookso_diary['name'] == ''].index
    sookso_diary.drop(indexNames, inplace=True) 



    if sookso_diary.like.dtype == 'O':
        sookso_diary.like = sookso_diary.like.apply(lambda x : convert(x))


    # 1, 중복된 이름 검색 
    # 2.개수가 1인거 이상 뽑아내기 
    # 3.리스트만들기
    sd_cnt = Counter(sookso_diary.name)      # Counter 모듈로 name data count
    li =[]
    for name, cnt  in sd_cnt.items():            # 중복된 것만 뽑기 (cnt가 1이상이면 중복)
        if cnt > 1:
            li.append(name)

    # 4. 행가져와서 like집계, content, tag list append
    for i in li:           # 중복된 name 반복
        ex = sookso_diary.loc[sookso_diary.name == i]  # 반복된 name의 행을 가져온다
        ex.reset_index(drop=True,inplace=True)     # index reset

        content = ex.iloc[0,:].content
        date = ex.iloc[0,:].date
        like = ex.iloc[0,:].like
        place = ex.iloc[0,:].place
        tags = ex.iloc[0,:].tags
        imgUrl = ex.iloc[0,:].imgUrl
        name = ex.name[0]

        for i in range(len(ex)-1):
            like += ex.iloc[i+1,:].like
            tags += ex.iloc[i+1,:].tags



        # 5.차례대로 기존 행 지워내기
        indexNames = sookso_diary[sookso_diary['name'] == ex.name[0]].index
        sookso_diary.drop(indexNames, inplace=True) 
        sookso_diary = sookso_diary.append(pd.Series(data=[content,date,like,place,tags,imgUrl,name],index=sookso_diary.columns),ignore_index=True)

    sd_cnt_tmp = []
    for i in sookso_diary.name:
        sd_cnt_tmp.append(sd_cnt[i]-1)

    sookso_diary['overlap'] = sd_cnt_tmp

    #####################################################

    ### sookso.hada (완료)
    sookso_hada_name = sookso_hada.content.apply(lambda x : sookso_hada_extract(x))
    sookso_hada['name'] = sookso_hada_name
    sookso_hada = sookso_hada[~(sookso_hada['name'].isna())]

    # name data가 없는 index 지우기
    indexNames = sookso_hada[sookso_hada['name'] == ''].index
    sookso_hada.drop(indexNames, inplace=True) 


    # like integet 변환 후 합치기

    if sookso_hada.like.dtype == 'O':
        sookso_hada.like = sookso_hada.like.apply(lambda x : convert(x))


    # 1, 중복된 이름 검색 
    # 2.개수가 1인거 이상 뽑아내기 
    # 3.리스트만들기
    sh_cnt = Counter(sookso_hada.name)      # Counter 모듈로 name data count
    li =[]
    for name, cnt  in sh_cnt.items():            # 중복된 것만 뽑기 (cnt가 1이상이면 중복)
        if cnt > 1:
            li.append(name)        


    # 4. 행가져와서 like집계, content, tag list append
    for i in li:           # 중복된 name 반복
        ex = sookso_hada.loc[sookso_hada.name == i]  # 반복된 name의 행을 가져온다
        ex.reset_index(drop=True,inplace=True)     # index reset

        content = ex.iloc[0,:].content
        date = ex.iloc[0,:].date
        like = ex.iloc[0,:].like
        place = ex.iloc[0,:].place
        tags = ex.iloc[0,:].tags
        imgUrl = ex.iloc[0,:].imgUrl
        name = ex.name[0]

        for i in range(len(ex)-1):
            like += ex.iloc[i+1,:].like
            tags += ex.iloc[i+1,:].tags

        # 5.차례대로 기존 행 지워내기
        indexNames = sookso_hada[sookso_hada['name'] == ex.name[0]].index
        sookso_hada.drop(indexNames, inplace=True) 
        sookso_hada = sookso_hada.append(pd.Series(data=[content,date,like,place,tags,imgUrl,name],index=sookso_hada.columns),ignore_index=True)

    sh_cnt_tmp = []
    for i in sookso_hada.name:
        sh_cnt_tmp.append(sh_cnt[i]-1)

    sookso_hada['overlap'] = sh_cnt_tmp

    ######################################
    ##  데이터 프레임 합치기
    tot_dataset = daily_gamsung
    for df in [gamsung_bnb, gamsung_curation, hi_stay_tour, rest_behappyhere, sookso_diary, sookso_hada]:
        tot_dataset = tot_dataset.append(df)

    if tot_dataset.like.dtype == 'O':
        tot_dataset.like = tot_dataset.like.apply(lambda x : convert(x))



    dg_cnt = Counter(tot_dataset.name)      # Counter 모듈로 name data count
    li =[]
    for name, cnt  in dg_cnt.items():            # 중복된 것만 뽑기 (cnt가 1이상이면 중복)
        if cnt > 1:
            li.append(name)


    # 4. 행가져와서 like집계, content, tag list append
    for i in li:           # 중복된 name 반복
        ex = tot_dataset.loc[tot_dataset.name == i]  # 반복된 name의 행을 가져온다
        ex.reset_index(drop=True,inplace=True)     # index reset

        content = ex.iloc[0,:].content
        date = ex.iloc[0,:].date
        like = ex.iloc[0,:].like
        place = ex.iloc[0,:].place
        tags = ex.iloc[0,:].tags
        imgUrl = ex.iloc[0,:].imgUrl
        name = ex.name[0]
        overlap = ex.iloc[0,:].overlap

        for i in range(len(ex)-1):
            like += ex.iloc[i+1,:].like
            tags += ex.iloc[i+1,:].tags
            overlap += ex.iloc[i+1,:].overlap


        # 5.차례대로 기존 행 지워내기
        indexNames = tot_dataset[tot_dataset['name'] == ex.name[0]].index
        tot_dataset.drop(indexNames, inplace=True) 
        tot_dataset = tot_dataset.append(pd.Series(data=[content,date,like,place,tags,imgUrl,name,overlap],index=tot_dataset.columns),ignore_index=True)

    ####################################
    ## Naver 장소 API

    # naver application clientID,secret
    client_id = "6E6mBjvxbdEgVk6Prqgd"
    client_secret = "0ZjWnYiBjr"


    # quote가 인코딩을 변경(UTF8로 url에 넘겨주기 위해서)
    # 내가 입력한 값을 기준으로 parse
    title_et = []
    for name in tot_dataset.name:
        query = urllib.parse.quote(f'숙소+" "+{name}')
        idx = 0 # 요청 개수
        display = 10 # 100개 단위로 가져온다
        start = 1
        end = 100
        tmp=''

        url = "https://openapi.naver.com/v1/search/local?query=" + query \
            + "&display=" + str(display) \
            + "&start=" + str(start)


        request = urllib.request.Request(url)
        request.add_header("X-Naver-Client-Id",client_id)
        request.add_header("X-Naver-Client-Secret",client_secret)
        # request 요청에 대한 결과를 response에 저장
        response = urllib.request.urlopen(request)
        rescode = response.getcode()
        if(rescode==200):
            response_body = response.read()
            response_dict = json.loads(response_body.decode('utf-8'))
            items = response_dict['items']
            for item_index in range(0,len(items)):
                remove_tag = re.compile('<.*?>')
                title = re.sub(remove_tag, '', items[item_index]['title'])
                link = items[item_index]['link']
                caterory = items[item_index]['category']
                description = re.sub(remove_tag, '',items[item_index]['description'])
                telephone = items[item_index]['telephone']
                address = items[item_index]['address']
                roadAddress = items[item_index]['roadAddress']
                mapx = items[item_index]['mapx']
                mapy = items[item_index]['mapy']

                # 이름이 없는 경우
                if not title :
                    title_et.append('')
                else: #'전통숙소' '호스텔''리조트''리조트부속건물'
                    if caterory[:2] in ['숙박'] :  # 카테고리가 숙박인 경우
                        title = title.replace(' ','')
                        name = name.replace(' ','')
                        ratio = SequenceMatcher(None, title, name).ratio()
                        if ratio >= 0.7 : # title.replace(' ','') in name.replace(' ','')
                            tmp = address

            if tmp : title_et.append(tmp)
            else: title_et.append('')

        else:
            print("Error Code:" + rescode)

        time.sleep(0.1)


    tot_dataset.drop('place',axis=1,inplace=True)
    tot_dataset['place'] = title_et

    #####################################
    ### 코로나 단계 작업
    coronaStage = coronaStage.toPandas() 

    stage =[str() for i in range(len(tot_dataset))]
    local = [str() for i in range(len(tot_dataset))]
    for lo in coronaStage.Area.unique().tolist():
        for j in range(len(tot_dataset)):
            try :
                com = tot_dataset.iloc[j].place.split()[0]
                if list(lo)[0] in com and list(lo)[1] in com:
                    stage[j] = coronaStage[coronaStage.Area == lo].Stage.values[0]
                    local[j] = lo
            except:
                pass

    tot_dataset['stage'] = stage
    tot_dataset['local'] = local

    ############################
    ## 새로운 숙소 생성


    ch = []
    for i in cur.find():
        ch.append(i)
    change = pd.DataFrame(ch)

    a = tot_dataset.merge(change,how='left' ,on='name')
    li = a[a.tags_y.isnull() == True].index.tolist()
    new = tot_dataset.iloc[li]
    print(new)
#     new.drop('index',axis=1,inplace=True)
    new.to_parquet(f'new_{nowtime}.parquet')

insta_spark_processing()

Empty DataFrame
Columns: [content, date, like, tags, imgUrl, name, overlap, place, stage, local]
Index: []


In [80]:
change['imgUrl'] = a['imgUrl_y']

In [90]:
change.drop('index',axis=1,inplace=True)

In [92]:
change.to_parquet(f'tot_dataset_{nowtime}.parquet')


---

### 클러스터링

In [26]:
insta_data = tot_dataset.copy()

In [27]:
from kiwipiepy import Kiwi

kiwi = Kiwi()
kiwi.prepare()

location = pd.read_csv('/home/ubuntu/DS/location_words.csv')
location.columns = ['index', 'location']
location = location.location.tolist()

def sub_special(s):
    rs = re.sub(r'[^가-힣]',' ',s)
    rr = re.sub(' +', ' ', rs)
    return rr

def tokenize_nouns(sent):
    res, score = kiwi.analyze(sent)[0] # 첫번째 결과를 사용
    return [word + ('다' if tag.startswith('V') else '') # 동사에는 '다'를 붙여줌
            for word, tag, _, _ in res
            if tag.startswith('N')] # 조사, 어미, 특수기호 및 stopwords에 포함된 단어는 제거

/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:4: DeprecationWarning: `prepare()` has no effect and will be removed in future version.


In [28]:
# 1차 클린징(영어, 특수문자, 숫자제거)
for i in range(len(insta_data.content)):
    insta_data.content[i] = sub_special(insta_data.content[i])

# 토큰화
for i in range(len(insta_data.content)):
    insta_data.content[i]= tokenize_nouns(insta_data.content[i])

/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [31]:
insta_data['location'] = np.NaN

In [91]:
stop_words = '감성,인,수,등,나,애정,상황,문의,작동,금,동,속,천장,석양,해,해안,남,추천,방송,우정,소원,동일,복,수,내,안전,석,가능,가치,동리,회,사방,궁,당,대,동막,사진,통,창,감,구성,점,시,전,조리,요리,조리,노천,온수,영지,길,장,후,입,시,원,가구,내부,오'
stop_words = stop_words.split(',')

In [92]:
for i in range(len(insta_data.content)):
    result = []
    for w in insta_data.content[i]:
        if w in location and w not in stop_words:
            result.append(w)
        insta_data.location[i] = result

/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [93]:
for i in range(len(insta_data.location)):
    temp = set(insta_data.location[i])
    insta_data.location[i] = list(temp)
    

/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [94]:
insta_data.location

0          [밀양, 밀양시, 산외면]
1                    [여수]
2      [화도면, 강화군, 강화, 인천]
3          [제주시, 구좌읍, 제주]
4           [전북, 전주, 전주시]
              ...        
694    [상관면, 완주, 완주군, 전주]
695        [강릉시, 강릉, 연곡면]
696    [중앙, 강릉시, 강릉, 사천면]
697        [포항, 포항시, 청하면]
698         [정방, 서귀포, 제주]
Name: location, Length: 699, dtype: object

In [ ]:
from datetime import datetime, timedelta
from pymongo import MongoClient
import pandas as pd 
import numpy as np
import json


# pymongo connect
client = MongoClient('localhost',27017) # mongodb 27017 port
db = client.ojo_db


In [ ]:
# # mognodb에 넣기위한 json 형태 변환 후 mongodb collection(corona)로 적재
# for k in range(len(tot_dataset)):
#     tmp ={}
#     for key,value in zip(tot_dataset.columns,tot_dataset.values[k]):
#         tmp[key] = value
#     db.insta_tot.insert_one(tmp)